# Homework

Download the model from here: 

In [ ]:
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/dino-dragon-model/dino_dragon_10_0.899.h5

Install Tenserflow Lite

In [ ]:
!pip install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime

## Question 1

Now convert this model from Keras to TF-Lite format.

What's the size of the **converted** model?

* 21 Mb
* 43 Mb
* 80 Mb
* 164 Mb

In [3]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

In [ ]:
model = keras.models.load_model('dino_dragon_10_0.899.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('dino_dragon_10_0.899.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

In [42]:
!ls -lh *.tflite

-rw-r--r-- 1 user user 43M Nov 23 06:52 dino_dragon_10_0.899.tflite


The tflite model has a size of `43Mb`.

## Question 2

To be able to use this model, we need to know the index of the input and 
the index of the output. 

What's the output index for this model?

* 3
* 7
* 13
* 24

In [4]:
import tensorflow.lite as tflite

In [6]:
interpreter = tflite.Interpreter(model_path='dino_dragon_10_0.899.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [7]:
output_index

13

## Preparing the image

You'll need some code for downloading and resizing images. You can use 
this code:

In [8]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.Resampling.NEAREST)
    return img

Let's download and resize this image: 

https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg

Based on the previous homework, what should be the target size for the image?

In [32]:
img = download_image('https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg')
img = prepare_image(img, target_size=(150, 150))

# reshape and rescale image
X = np.divide(img, 255, dtype='float32').reshape((1, 150, 150, 3))

## Question 3

Now we need to turn the image into numpy array and pre-process it. 

> Tip: Check the previous homework. What was the pre-processing 
> we did there?

After the pre-processing, what's the value in the first pixel, the R channel?

* 0.3353411
* 0.5529412
* 0.7458824
* 0.9654902


In [33]:
X[0][0][0][0]

0.5529412

## Question 4

Now let's apply this model to this image. What's the output of the model?

* 0.17049132
* 0.39009996
* 0.60146114
* 0.82448614

In [57]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [60]:
preds[0][0]

0.82448506

## Prepare the lambda code 

Now you need to copy all the code into a separate python file. You will 
need to use this file for the next two questions.

Tip: you can test this file locally with `ipython` or Jupyter Notebook 
by importing the file and invoking the function from this file.  


## Docker 

For the next two questions, we'll use a Docker image that we already 
prepared. This is the Dockerfile that we used for creating the image:

```docker
FROM public.ecr.aws/lambda/python:3.9
COPY dino-vs-dragon-v2.tflite .
```

And pushed it to [`svizor42/zoomcamp-dino-dragon-lambda:v2`](https://hub.docker.com/r/svizor42/zoomcamp-dino-dragon-lambda/tags).


> Note: The image already contains a model and it's not the same model
> as the one we used for questions 1-4.


## Question 5

Download the base image `svizor42/zoomcamp-dino-dragon-lambda:v2`. You can easily make it by using [docker pull](https://docs.docker.com/engine/reference/commandline/pull/) command.

So what's the size of this base image?

* 139 Mb
* 329 Mb
* 639 Mb
* 929 Mb

You can get this information when running `docker images` - it'll be in the "SIZE" column.

In [ ]:
!docker image pull svizor42/zoomcamp-dino-dragon-lambda:v2

In [29]:
!docker image ls svizor42/zoomcamp-dino-dragon-lambda

REPOSITORY                             TAG       IMAGE ID       CREATED      SIZE
svizor42/zoomcamp-dino-dragon-lambda   v2        20ef58b21a05   5 days ago   639MB


## Question 6

Now let's extend this docker image, install all the required libraries
and add the code for lambda.

You don't need to include the model in the image. It's already included. 
The name of the file with the model is `dino-vs-dragon-v2.tflite` and it's 
in the current workdir in the image (see the Dockerfile above for the 
reference).

Now run the container locally.

Score this image: https://upload.wikimedia.org/wikipedia/en/e/e9/GodzillaEncounterModel.jpg

What's the output from the model?

* 0.12
* 0.32
* 0.52
* 0.72

In [ ]:
!docker build -t dino-dragon .

In [ ]:
!docker run -d --rm -p 8080:8080 dino-dragon:latest

In [61]:
import requests

url = 'http://localhost:8080/2015-03-31/functions/function/invocations'

data = {'url': 'https://upload.wikimedia.org/wikipedia/en/e/e9/GodzillaEncounterModel.jpg'}

result = requests.post(url, json=data).json()
print(result)

{'probability_for_dragon': 0.31950676441192627, 'prediction': 'dino'}
